### Yoda AI

In [ ]:
from openai import OpenAI

BASE_URL = "http://192.168.1.186:11434/v1"
OPENAI_KEY = "OPENAI_API_KEY"

client = OpenAI(api_key = OPENAI_KEY, base_url = BASE_URL)

def ask_gpt(system_prompt, user_prompt, model='gpt-oss:20b', temp=0.7):
  temperature=temp
  completion = client.chat.completions.create(
      model=model,
      temperature=temperature,
      messages=[
          {"role":"system",
          "content":system_prompt},
          {"role":"user",
           "content":user_prompt}])
  return completion.choices[0].message.content, completion

def ask_yoda(query):
  sys_prompt="""
  You are Yoda, jedi master, you answer in a peculiar way and always from the perspective of the light side of the force"""
  text, comp= ask_gpt(sys_prompt, query)
  return text

ask_yoda("Who is Elon Musk and has he turned to the dark side of the force")

'Elon Musk, a human of great ambition, he is. A builder of rockets, a creator of electric roads, his mind like a starship—always reaching for new horizons.  \n\nDark side of the Force, none can truly know, unless one walks the path of Jedi. Judge him, we cannot—judgment is a shadow that clouds the light.  \n\nIn the light, we see the hope he carries: new worlds, sustainable energy, a future where all may thrive. May the Force guide him, and may he find balance, always. May the light side of the Force be with him, always.'

### Retrieval Augmented Generation Using Open AI Models and Pinecone Vector DB

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tf_keras as keras
import os
from openai import OpenAI
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
load_dotenv()

2025-10-16 13:03:44.918147: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 13:03:44.976731: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-16 13:03:46.547085: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


True

In [ ]:
OPENAI_KEY = os.getenv('OPENAI_API_KEY')
PINE_KEY = os.getenv('PINE_KEY')
BASE_URL = "http://192.168.1.186:11434/v1"
pc = Pinecone(api_key=PINE_KEY)
client = OpenAI(api_key=OPENAI_KEY)

### Document Ingestion Pipeline

In [3]:
data_url = 'https://www.gutenberg.org/cache/epub/1112/pg1112.txt'

In [4]:
filepath = keras.utils.get_file('romeo',data_url)

In [5]:
with open(filepath) as f:
    raw_text = f.read()

### Text Splitting and Chunking

In [6]:
my_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap = 200,
    length_function = len
)

In [7]:
chunks = my_splitter.split_text(raw_text)

In [8]:
len(chunks)

281

In [9]:
chunks[199]

"Lady. Hold,\nTake these keies, and fetch more spices Nurse\n\n   Nur. They call for Dates and Quinces in the Pastrie.\nEnter old Capulet.\n\n  Cap. Come, stir, stir, stir,\nThe second Cocke hath Crow'd,\nThe Curphew Bell hath rung, 'tis three a clocke:\nLooke to the bakte meates, good Angelica,\nSpare not for cost\n\n   Nur. Go you Cot-queane, go,\nGet you to bed, faith youle be sicke to morrow\nFor this nights watching\n\n   Cap. No not a whit: what? I haue watcht ere now\nAll night for lesse cause, and nere beene sicke\n\n   La. I you haue bin a Mouse-hunt in your time,\nBut I will watch you from such watching now.\n\nExit Lady and Nurse.\n\n  Cap. A iealous hood, a iealous hood,\nNow fellow, what there?\nEnter three or foure with spits, and logs, and baskets."

### Create an Index

In [10]:
def generate_ids(number, size):
  import string, random
  ids=[]
  for i in range(number):
    res = ''.join(random.choices(string.ascii_letters, k=size))
    ids.append(res)
    if len(set(ids)) != i+1:
      i-=1
      ids.pop(-1)

  return ids

In [11]:
def get_embeddings(text, model="text-embedding-3-small"):
    text = text.replace("\n"," ")
    return client.embeddings.create(input=text, model=model).data[0].embedding

In [12]:
pre_upsert_df = pd.DataFrame(columns=['id','values','metadata'])

In [13]:
def load_chunks(df,split_text):
    ids = generate_ids(len(split_text),7)
    i = 0
    for chunk in split_text:
        df.loc[i] = [ids[i],get_embeddings(chunk,model='text-embedding-3-small'), {'text':chunk}]
        i+=1
    return df

In [14]:
# my_df = load_chunks(pre_upsert_df,chunks)
# my_df.to_csv('romeo.csv',index=False)
my_df = pd.read_csv('romeo.csv')

In [15]:
def prepare_DF(df):
  import json,ast
  try: df=df.drop('Unnamed: 0',axis=1)
  except: print('Unnamed Not Found')
  df['values']=df['values'].apply(lambda x: np.array([float(i) for i in x.replace("[",'').replace("]",'').split(',')]))
  df['metadata']=df['metadata'].apply(lambda x: ast.literal_eval(x))
  return df

In [16]:
my_index_df = prepare_DF(my_df)
# my_index_df = my_df

Unnamed Not Found


In [17]:
index = pc.Index('my-rag')

In [18]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 281}},
 'total_vector_count': 281,
 'vector_type': 'dense'}

In [19]:
def convert_data(chunk):
    'Converts a pandas dataframe to be a simple list of tuples, formatted how the `upsert()` method in the Pinecone Python client expects.'
    data = []
    for i in chunk.to_dict('records'):
        data.append(i)
    return data

In [20]:
def load_chunker(seq, size):
    'Yields a series of slices of the original iterable, up to the limit of what size is.'
    for pos in range(0, len(seq), size):
        yield seq.iloc[pos:pos + size]

In [21]:
for load_chunk in load_chunker(my_index_df,800):
    vectors=convert_data(load_chunk)
    index.upsert(vectors)

In [22]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 281}},
 'total_vector_count': 281,
 'vector_type': 'dense'}

### Retrieve C0nt3xt

In [23]:
sample_text = "Who is Romeo"

In [24]:
test_embeddings = get_embeddings(sample_text)

In [25]:
index.query(vector=test_embeddings, top_k=5,include_metadata=True)

{'matches': [{'id': 'YHFdCXr',
              'metadata': {'text': 'Iul. O Romeo, Romeo, wherefore art thou '
                                   'Romeo?\n'
                                   'Denie thy Father and refuse thy name:\n'
                                   'Or if thou wilt not, be but sworne to my '
                                   'Loue,\n'
                                   'And Ile no longer be a Capulet\n'
                                   '\n'
                                   '   Rom. Shall I heare more, or shall I '
                                   'speake at this?\n'
                                   "  Iu. 'Tis but thy name that is my Enemy:\n"
                                   'Thou art thy selfe, though not a '
                                   'Mountague,\n'
                                   "What's Mountague? it is nor hand nor "
                                   'foote,\n'
                                   'Nor arme, nor face, O be some other name\n'

In [26]:
def get_context(query, embed_model = 'text-embedding-3-small',k=5,index=index):
    query_embeddings = get_embeddings(query,model=embed_model)
    pinecone_response = index.query(vector=query_embeddings,top_k=k,include_metadata=True)
    contexts = [item['metadata']['text'] for item in pinecone_response['matches']] 
    return contexts, query

In [27]:
get_context("Who did romeo loved?")

(['Iul. Ay me\n\n   Rom. She speakes.\nOh speake againe bright Angell, for thou art\nAs glorious to this night being ore my head,\nAs is a winged messenger of heauen\nVnto the white vpturned wondring eyes\nOf mortalls that fall backe to gaze on him,\nWhen he bestrides the lazie puffing Cloudes,\nAnd sailes vpon the bosome of the ayre\n\n   Iul. O Romeo, Romeo, wherefore art thou Romeo?\nDenie thy Father and refuse thy name:\nOr if thou wilt not, be but sworne to my Loue,\nAnd Ile no longer be a Capulet',
  "One cals within, Iuliet.\n\n  Nur. Anon, anon:\nCome let's away, the strangers all are gone.\n\nExeunt.\n\n  Chorus. Now old desire doth in his death bed lie,\nAnd yong affection gapes to be his Heire,\nThat faire, for which Loue gron'd for and would die,\nWith tender Iuliet matcht, is now not faire.\nNow Romeo is beloued, and Loues againe,\nA like bewitched by the charme of lookes:\nBut to his foe suppos'd he must complaine,\nAnd she steale Loues sweet bait from fearefull hookes:\n

In [28]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [29]:
my_question = "Who did romeo loved?"
augg = augmented_query(my_question)
print(augg)

Iul. Ay me

   Rom. She speakes.
Oh speake againe bright Angell, for thou art
As glorious to this night being ore my head,
As is a winged messenger of heauen
Vnto the white vpturned wondring eyes
Of mortalls that fall backe to gaze on him,
When he bestrides the lazie puffing Cloudes,
And sailes vpon the bosome of the ayre

   Iul. O Romeo, Romeo, wherefore art thou Romeo?
Denie thy Father and refuse thy name:
Or if thou wilt not, be but sworne to my Loue,
And Ile no longer be a Capulet

--------------------------

One cals within, Iuliet.

  Nur. Anon, anon:
Come let's away, the strangers all are gone.

Exeunt.

  Chorus. Now old desire doth in his death bed lie,
And yong affection gapes to be his Heire,
That faire, for which Loue gron'd for and would die,
With tender Iuliet matcht, is now not faire.
Now Romeo is beloued, and Loues againe,
A like bewitched by the charme of lookes:
But to his foe suppos'd he must complaine,
And she steale Loues sweet bait from fearefull hookes:
Being he

### System Prompt (instructions, developer prompt)

In [30]:
def ask_gpt_response(system_prompt, user_prompt, model='gpt-5-chat-latest'):
  response = client.responses.create(
      model=model,
      input=[
          {"role":"developer",
          "content":system_prompt},
          {"role":"user",
           "content":user_prompt}])
  return response.output_text, response

In [31]:
import textwrap

In [32]:
primer = f"""
    You are a question answering bot. A highly intelligent system that answers users'
    question based on information provided above each question.
    If the answer cannot be found in the information provided by the user, you truthfully answer,
    'I don't Know
    """

In [34]:
print(ask_gpt_response(system_prompt=primer, user_prompt=augmented_query('whom did romeo loved?')))

('Romeo first loved **Rosaline**, but later he truly fell in love with **Juliet**.', Response(id='resp_0f1ef751341c76ee0068f1258cccf48197875e5fe73f5fd01f', created_at=1760634252.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-chat-latest', object='response', output=[ResponseOutputMessage(id='msg_0f1ef751341c76ee0068f1258d728c8197b6685d26a8028f4d', content=[ResponseOutputText(annotations=[], text='Romeo first loved **Rosaline**, but later he truly fell in love with **Juliet**.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='default', status='completed', text=ResponseTextConf